# Instlação

* pip install --upgrade pip
* pip install kafka-python
* pip install newsapi-python
* pip install schedule

# Bibliotecas

In [0]:
import pyspark.pandas as ps
import pandas as pd
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.functions import to_date, col
from kafka import KafkaConsumer
import datetime
import pickle

# Pipeline

In [0]:
class Data_Pipeline_API:
    def __init__(self) -> None:
        self.__df = None
        self.__df_parquet = None


    # Carregar dados da API
    def load_api(self, data) -> None:
        self.__df = ps.DataFrame(data)
        sources = []
        for aux in self.__df['source'].to_numpy():
            if isinstance(aux, dict):
                sources.append(aux['name'])
            else:
                sources.append(None)
        self.__df['source'] = sources


    # Transformar os dados .parquet em DataFrame
    def load_parquet(self, name) -> None:
        file = f"{name}.parquet"
        self.__df_parquet = ps.read_parquet(f"/FileStore/Projeto/data/{file}")


    # Concatenando o DataFrame com o arquivo parquet
    # Mantém apenas colunas únicas
    def save_data(self, name) -> None:
        file = f"{name}.parquet"
        if file in [y.name.strip() for y in dbutils.fs.ls("/FileStore/Projeto/data/")] or f"{file}/" in [y.name.strip() for y in dbutils.fs.ls("/FileStore/Projeto/data/")]:
            self.load_parquet(name)
            self.__df = ps.concat([self.__df_parquet, self.__df]).drop_duplicates().reset_index().drop(columns=["index"])
        self.__df.to_parquet(f"/FileStore/Projeto/data/{file}", mode='overwrite')


    # Limpeza dos dados Brutos
    def data_cleaning(self):
        # Remove coluna de url de imagem de capa
        self.__df = self.__df_parquet.drop(columns=['urlToImage'])
        
        #Alterar o nome das colunas
        self.__df = self.__df.rename(columns={'author': 'autor',
                                              'title': 'titulo',
                                              'description': 'descricao',
                                              'publishedAt': 'data_publicacao',
                                              'content': 'conteudo',
                                              'source': 'fonte'})

        # Organizar colunas
        df_order = ['data_publicacao', 'titulo', 'autor', 'descricao', 'url', 'fonte', 'conteudo']
        self.__df = self.__df[df_order]

        # Ajuste do dia de publicação
        data = self.__df['data_publicacao'].to_numpy()
        date_df = pd.DataFrame(data = data)
        date_df.columns = ['data_publicacao']
        date_df['data_publicacao'] = pd.to_datetime(date_df['data_publicacao'])
        date_df['data_publicacao'] = date_df['data_publicacao'].dt.strftime('%Y/%m/%d')
        
        # Atribuição de valores
        for i in range(self.__df.shape[0]):
            self.__df.loc[i, 'data_publicacao'] = date_df.loc[i, 'data_publicacao']

        # Ordenar Datas
        self.__df = self.__df.sort_values('data_publicacao').reset_index().drop(columns=['index'])


    # Salvando em Delta Table
    def delta_table(self):
        df_delta = spark.read.parquet("/FileStore/Projeto/data/cleaned_data.parquet")
        df_delta = df_delta.withColumn("data_publicacao", to_date(col("data_publicacao"), "yyyy/MM/dd"))

        # Salvando em Delta
        df_delta.write.format("delta").mode("overwrite").save("/FileStore/Projeto/delta")

        # Disponibilizando em banco de dados
        df_delta.write.format("delta").mode("overwrite").saveAsTable("pipeline_delta")
        

    @property
    def df(self):
        return self.__df
    
    @df.setter
    def df(self, dataframe):
        self.__df = dataframe


    @property
    def df_parquet(self):
        return self.__df_parquet

# Consumer

In [0]:
consumer = KafkaConsumer("genomics-news", bootstrap_servers=["localhost:9092"])

for message in consumer:
    now = datetime.datetime.now()
    now_min = now.minute
    print(f"Processo iniciado em {now.strftime('%Y-%m-%d %H:%M:%S')}")
    
    value = message.value
    articles_list = pickle.loads(value)

    if len(articles_list) != 0:
        
        # Instanciando Pipeline
        pipeline_data = Data_Pipeline_API()

        # Carregando dados da API
        pipeline_data.load_api(articles_list)

        # Salvando em Parquet
        pipeline_data.save_data("raw_data")
        print(f"raw_data atualizado em {now.strftime('%Y-%m-%d %H:%M:%S')}")

  File <command-898858298568835>, line 60
    self.__df. = self.__df.withColumn("data_publicacao", to_date(df["data_publicacao"], "yyyy/MM/dd"))
               ^
SyntaxError: invalid syntax
